In [1]:
%pip install beautifulsoup4
# %pip install nltk
# %pip install spacy
# python -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# %pip install sparknlp
# %pip install pyspark

In [11]:
%pip install findspark


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
%pip install spark-nlp==3.3.0

     ---------------------------------------- 0.0/120.6 kB ? eta -:--:--
     ----------------------------- --------- 92.2/120.6 kB 1.7 MB/s eta 0:00:01
     -------------------------------------- 120.6/120.6 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: spark-nlp
    Found existing installation: spark-nlp 5.1.4
    Uninstalling spark-nlp-5.1.4:
      Successfully uninstalled spark-nlp-5.1.4
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
%pip install spark-nlp --upgrade

  Using cached spark_nlp-5.1.4-py2.py3-none-any.whl (540 kB)
  Attempting uninstall: spark-nlp
    Found existing installation: spark-nlp 3.3.0
    Uninstalling spark-nlp-3.3.0:
      Successfully uninstalled spark-nlp-3.3.0
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import requests
from bs4 import BeautifulSoup
import re

import concurrent.futures

In [ ]:
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from nltk import pos_tag

import numpy as np

In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nltk.download('wordnet')

In [29]:
wnl = WordNetLemmatizer()

def isplural(word):
    lemma = wnl.lemmatize(word, 'n')
    plural = True if word is not lemma else False
    return plural, lemma

In [5]:
with open('furniture stores pages.csv', 'r') as f:
    lines = f.readlines()
    urls = lines[1:]

urls = list(map(lambda x : re.sub(r'\n', '', x), urls))

In [8]:
%%time
def scrape_product_info(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
    
        # Extract product information based on the HTML structure of the websites
        try:
            title = soup.find('title').text.strip()
        except:
            title = ""
        try:
            description = soup.find('meta', {'name': 'description'})['content'].strip()
        except:
            description = ""

        response = {'title': title, 'description': description, 'url': url}
    except:
        response = {}

    # You can add more logic to extract other relevant information

    return response

def scrape_product_info_parallel(urls, max_workers=None):
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Use map to apply the square_number function to each element of the list
        squared_numbers = list(executor.map(scrape_product_info, urls))
    return squared_numbers

product_info = scrape_product_info_parallel(urls)

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


CPU times: user 2min 51s, sys: 5.62 s, total: 2min 57s
Wall time: 6min 9s


In [9]:
data = [v.get('title', '') + v.get('description', '') for v in product_info]

In [10]:
data[:10]

['Factory Buys 32cm Euro Top Mattress - KingPamper yourself with Factory Buys Euro Top Mattress that will give you a soothing sleep night after night.Incorporating the cutting-edge technology, the lavish construction addresses all elements that will offer you a comfortable and luxurious experience - Euro top padding with high-density foam and an independent coil',
 'Beadlight Cirrus LED Reading Light\n– Dunlin HomeBeadlight wall light, LED reading light, First call lights, reading light first class, plane reading lights, reading light led, bed light, beadhead light, led',
 'Hamar Plant Stand - Ash\n– The ModernThe Hamar Plant Stand is a wonderful accessory that holds at least three pot plants or other objects such as vases, sculptural objects or lamps. Use it to fill a corner or that tricky spot, bring height and greenery to your room.\xa0\xa0 Dimensions: 65cm at widest point x 60cm (D) x 92cm (H)Each platform has 28cm (W) x 28cm',
 '',
 '',
 '404 Not Found – Home-Buy Interiors',
 'Gif

In [48]:
# Tokenize the text into words
all_words = []
for text in data:
    words = word_tokenize(text.lower())  # Convert to lowercase to ensure case-insensitivity
    all_words.extend(words)

filtered_words = [word for word in all_words if word not in stop_words]

In [49]:
pos_tags = pos_tag(filtered_words)

In [50]:
nouns = [word for word, pos in pos_tags if pos.startswith('NN')]

In [51]:
lemmatized_nouns = [wnl.lemmatize(word, 'n') for word in nouns]

In [149]:
nltk.download('words')
english_words = set(nltk.corpus.words.words())

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Andrei\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


In [150]:
english_words = [word.lower() for word in lemmatized_nouns if word.lower() in english_words]

In [151]:
unique_words, counts = np.unique(english_words, return_counts=True)

In [152]:
sorted_word_frequency = {k: v for (k,v) in sorted(list(zip(unique_words, counts)), key=lambda item: item[1], reverse=True) if len(k)>2}

In [156]:
import csv

In [157]:
# Write the list to the CSV file
with open('annotated_words.csv', "w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["Words"])  # Write header
    csv_writer.writerows([[element] for element in sorted_word_frequency.keys()])  # Write elements


In [1]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

import sparknlp
spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.1.4
Apache Spark version:  3.5.0


In [2]:
import sparknlp
from sparknlp.annotator import *
from sparknlp. training import CoNLL
from pyspark.ml import Pipeline
spark = sparknlp.start()

from pyspark.sql import SparkSession
from sparknlp.annotator import Tokenizer, PerceptronModel, Lemmatizer, StopWordsCleaner
from sparknlp import DocumentAssembler
from sparknlp.base import Finisher

In [ ]:
training_data = CoNLL().readDataset(spark, './eng.train' ) # CONLL2003 train set

bert = BertEmbeddings.pretrained('bert_base_cased', 'en')\
        .setInputCols(["sentence", 'token'])\
        .setOutputCol("bert")\

# Trains Tensorflow based Char-CNN-BLSTM model
nerTagger = NerDLApproach()\
    .setInputCols(["sentence", "token", "bert"])\
    .setLabelColumn( "label" ) \
    .setOutputCol("ner")\
    .setMaxEpochs(1)

pipeline = Pipeline(stages = [bert, nerTagger])
nermodel = pipeline.fit(training_data)

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from sparknlp.base import DocumentAssembler, Finisher
from sparknlp.annotator import Tokenizer, PerceptronModel, Lemmatizer, StopWordsCleaner

import sparknlp
spark = sparknlp.start()

In [3]:
from pyspark.ml import Pipeline

In [44]:
# Create a Spark session
spark = SparkSession.builder \
    .appName("Spark NLP Tokenizer Example") \
    .getOrCreate()

# Sample data
data = [("1", "Spark NLP is an awesome library. Natural language processing made easy with Spark."),
               # 'B-PROD, I-PROD, O, O, B-FEAT, B-PROD, O, B-PROD, I-PROD, I-PROD, O, O, O, B-PROD, O'),
        ("2", "Natural language processing made easy with Spark. Spark NLP is an awesome library."),
                # 'B-PROD, I-PROD, I-PROD, O, O, O, B-PROD, O, B-PROD, I-PROD, O, O, B-FEAT, B-PROD, O')
       ]

# Create a DataFrame
columns = ["id", "text"] #, "label"]
df = spark.createDataFrame(data, columns)

In [4]:
# Define the pipeline
document_assembler = DocumentAssembler().setInputCol("text").setOutputCol("document")
tokenizer = Tokenizer().setInputCols(["document"]).setOutputCol("token")
finisher = Finisher().setInputCols(["token"]).setOutputCols(["tokens"]).setOutputAsArray(True)

In [6]:
# Assemble the pipeline
pipeline = [document_assembler, tokenizer, finisher]
pipeline = Pipeline().setStages(pipeline)

In [7]:
# Fit and transform the DataFrame
result = pipeline.fit(df).transform(df)

In [8]:
# Show the result
result.select("id", "tokens").show(truncate=False)

+---+----------------------------------------------------------------------------------------------------+
|id |tokens                                                                                              |
+---+----------------------------------------------------------------------------------------------------+
|1  |[Spark, NLP, is, an, awesome, library, ., Natural, language, processing, made, easy, with, Spark, .]|
|2  |[Natural, language, processing, made, easy, with, Spark, ., Spark, NLP, is, an, awesome, library, .]|
+---+----------------------------------------------------------------------------------------------------+



In [11]:
# Define the pipeline
document_assembler = DocumentAssembler().setInputCol("text").setOutputCol("document")
tokenizer = Tokenizer().setInputCols(["document"]).setOutputCol("token")
pos_tagger = PerceptronModel.pretrained().setInputCols(["document", "token"]).setOutputCol("pos")
finisher = Finisher().setInputCols(["pos"]).setOutputCols(["pos_tags"]).setOutputAsArray(True)


pos_anc download started this may take some time.
Approximate size to download 3.9 MB
[OK!]


In [14]:
# Assemble the pipeline
pos_pipeline = [document_assembler, tokenizer, pos_tagger, finisher]
pos_pipeline = Pipeline().setStages(pos_pipeline)

In [15]:
# Fit and transform the DataFrame
result = pos_pipeline.fit(df).transform(df)

# Show the result
result.select("id", "text", "pos_tags").show(truncate=False)

+---+----------------------------------------------------------------------------------+----------------------------------------------------------------+
|id |text                                                                              |pos_tags                                                        |
+---+----------------------------------------------------------------------------------+----------------------------------------------------------------+
|1  |Spark NLP is an awesome library. Natural language processing made easy with Spark.|[NNP, NNP, VBZ, DT, JJ, NN, ., NNP, NN, NN, VBN, JJ, IN, NNP, .]|
|2  |Natural language processing made easy with Spark. Spark NLP is an awesome library.|[NNP, NN, NN, VBN, JJ, IN, NNP, ., NNP, NNP, VBZ, DT, JJ, NN, .]|
+---+----------------------------------------------------------------------------------+----------------------------------------------------------------+



In [34]:
from sparknlp.base import DocumentAssembler, Finisher

from sparknlp.annotator import (
    NerCrfApproach, 
    WordEmbeddingsModel,
    SentenceDetector,
    Tokenizer,
    PerceptronModel
)

In [35]:
from sparknlp.annotator import NerDLApproach

In [36]:
# Define the pipeline
document_assembler = DocumentAssembler() \
                        .setInputCol("text") \
                        .setOutputCol("document")
sentence_detector = SentenceDetector() \
                        .setInputCols("document") \
                        .setOutputCol("sentence")
tokenizer = Tokenizer() \
                        .setInputCols(["document"]) \
                        .setOutputCol("token")
pos_tagger = PerceptronModel.pretrained() \
                        .setInputCols("sentence", "token") \
                        .setOutputCol("pos")
embeddings = WordEmbeddingsModel.pretrained() \
                        .setInputCols("sentence", "token") \
                        .setOutputCol("embeddings") \
                        .setCaseSensitive(False)
ner_tagger = NerCrfApproach() \
                        .setInputCols("sentence", "token", "pos", "embeddings") \
                        .setLabelColumn("label") \
                        .setOutputCol("ner") \
                        .setMinEpochs(1) \
                        .setMaxEpochs(5) \
                        .setLossEps(1e-3)

finisher = Finisher().setInputCols(["ner"]).setOutputCols(["ner_tags"]).setOutputAsArray(True)

# Assemble the pipeline
ner_pipeline = [document_assembler, 
                sentence_detector, 
                tokenizer,
                pos_tagger,
                embeddings,
                ner_tagger,
                finisher
               ]

ner_pipeline = Pipeline().setStages(ner_pipeline)

pos_anc download started this may take some time.
Approximate size to download 3.9 MB
[OK!]
glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


In [38]:
# Fit and transform the DataFrame
result = ner_pipeline.fit(df).transform(df)

Py4JJavaError: An error occurred while calling o1344.fit.
: java.lang.ClassCastException: class java.lang.String cannot be cast to class scala.collection.TraversableLike (java.lang.String is in module java.base of loader 'bootstrap'; scala.collection.TraversableLike is in unnamed module of loader 'app')
	at com.johnsnowlabs.nlp.annotators.common.Tagged.getAnnotations(Tagged.scala:117)
	at com.johnsnowlabs.nlp.annotators.common.Tagged.getAnnotations$(Tagged.scala:116)
	at com.johnsnowlabs.nlp.annotators.common.NerTagged$.getAnnotations(Tagged.scala:165)
	at com.johnsnowlabs.nlp.annotators.common.NerTagged$.$anonfun$collectTrainingInstancesWithPos$1(Tagged.scala:180)
	at scala.collection.TraversableLike.$anonfun$flatMap$1(TraversableLike.scala:293)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:198)
	at

In [ ]:
# Show the result
result.select("id", "text", "ner_tags").show(truncate=False)

In [39]:
import json
import pandas as pd
import numpy as np

import sparknlp
import pyspark.sql.functions as F

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.types import StringType, IntegerType

In [42]:
documentAssembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

sentence_detector = SentenceDetector() \
    .setInputCols('document') \
    .setOutputCol('sentence')


tokenizer = Tokenizer() \
    .setInputCols(['sentence']) \
    .setOutputCol('token')

'''
Note:

For embeddings that offer multi-language support like glove_6B_300, we use 'xx'
in the language parameter.

e.g: embeddings = WordEmbeddingsModel.pretrained('glove_6B_300', lang='xx') \
        .setInputCols(["sentence", 'token']) \
        .setOutputCol("embeddings")

For dedicated embeddings like 'aner_cc_300d' the language parameter would be 
the corresponding language.

e.g: embeddings = WordEmbeddingsModel.pretrained('aner_cc_300d', lang='ar') \
        .setInputCols(["sentence", 'token']) \
        .setOutputCol("embeddings")

'''
## to use Glove embeddings
embeddings = WordEmbeddingsModel.pretrained('glove_6B_300', lang='xx') \
        .setInputCols(["sentence", 'token']) \
        .setOutputCol("embeddings")

## to use Bert Embeddings
#embeddings = BertEmbeddings.pretrained(name='bert_base_cased', lang='en') \
#        .setInputCols(['document', 'token']) \
#        .setOutputCol('embeddings')

# select your desired model and define language
ner_model = NerDLModel.pretrained() \
    .setInputCols(['sentence', 'token', 'embeddings']) \
    .setOutputCol('ner')

ner_converter = NerConverter() \
    .setInputCols(['sentence', 'token', 'ner']) \
    .setOutputCol('ner_chunk')

nlp_pipeline = Pipeline(
    stages=[
        documentAssembler, 
        sentence_detector,
        tokenizer,
        embeddings,
        ner_model,
        ner_converter])

glove_6B_300 download started this may take some time.
Approximate size to download 426.2 MB
[OK!]
ner_dl download started this may take some time.
Approximate size to download 13.6 MB
[OK!]


In [45]:
df = spark.createDataFrame(data, StringType()).toDF("text")
result = nlp_pipeline.fit(df).transform(df)

IllegalArgumentException: requirement failed: Found input column with storage metadata. But such ref does not match to the ref this annotator requires. Make sure you are loading the annotator with ref: glove_100d